# recursive directory object...

In [1]:
import os
import yt 

def make_filename_safe(fname):
    safename = fname.replace(".", "_").replace("-", "_")    
    if safename[0].isnumeric():
        safename = "_" + safename
    return safename

class FileLikeHelper:
    def __init__(self, pathlike):
        # print("setting fullpath")
        self._fullpath = os.path.abspath(pathlike)
        bname = os.path.basename(self._fullpath)
        base, ext = os.path.splitext(bname)
        
        self._name = os.path.basename(str(self._fullpath))
        self._safe_name = make_filename_safe(self._name)
        
    def __str__(self):
        return str(self._fullpath)
    
    def __repr__(self):
        return self._fullpath    

class FileInfo(FileLikeHelper):

    @property
    def filepath(self):
        return self._fullpath

class DirectoryInfo(FileLikeHelper):
    def __init__(self, pathlike, top_path, max_recursion_level = 10):   
        # print("hello")
        super().__init__(pathlike)
        new_depth = True    
        self.___files = []
        self.___directories = []
        for file_or_dir in os.listdir(self._fullpath):  
            
            fullfidir = os.path.join(self._fullpath, file_or_dir)
            if os.path.isfile(fullfidir):
                # print(f"adding {fullfidir} as file")
                newfile = FileInfo(fullfidir)
                setattr(self, newfile._safe_name, newfile)
                self.___files.append(file_or_dir)
            elif os.path.isdir(fullfidir):          
                # print(f"adding {fullfidir} as dir")
                new_dir_path = os.path.join(self._fullpath, fullfidir)
                rel_to_top = os.path.relpath(os.path.abspath(top_path), new_dir_path)
                self.___directories.append(file_or_dir)
                dist_to_top = len(rel_to_top.split(os.path.pathsep))
                if dist_to_top <= max_recursion_level:    
                    newdir = DirectoryInfo(new_dir_path, top_path, max_recursion_level=max_recursion_level)
                    setattr(self, newdir._safe_name, newdir)
        self.___files.sort()
        self.___directories.sort()
        
    def __repr__(self):
        finfi = f"Directory: {self._fullpath}\n\n"
        finfi = finfi + "Files:\n"
        for fi in self.___files:
            finfi += f"{fi}\n"
        finfi+="\nSubdirectories:\n"
            
        for thisdir in self.___directories:
            finfi += f"{thisdir}\n"
        
        return finfi 
    
def walk_directory(starting_directory, max_levels=10):    
    return DirectoryInfo(starting_directory, starting_directory, max_recursion_level=max_levels)

def walk_yt_data_directory(max_levels=30):    
    return walk_directory(yt.config.ytcfg.get('yt','test_data_dir'), max_levels=max_levels)

In [2]:
ytData = walk_yt_data_directory()

In [4]:
ytData.enzo_cosmology_plus.DD0000.DD0000

/home/chavlin/hdd/data/yt_data/yt_sample_sets/enzo_cosmology_plus/DD0000/DD0000

In [5]:
ds = yt.load(ytData.enzo_cosmology_plus.DD0000.DD0000.filepath)

yt : [INFO     ] 2023-03-23 16:30:23,826 Parameters: current_time              = 0.81651036015641
yt : [INFO     ] 2023-03-23 16:30:23,826 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2023-03-23 16:30:23,827 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-03-23 16:30:23,828 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2023-03-23 16:30:23,828 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2023-03-23 16:30:23,828 Parameters: current_redshift          = 50
yt : [INFO     ] 2023-03-23 16:30:23,829 Parameters: omega_lambda              = 0.732
yt : [INFO     ] 2023-03-23 16:30:23,829 Parameters: omega_matter              = 0.268
yt : [INFO     ] 2023-03-23 16:30:23,829 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2023-03-23 16:30:23,830 Parameters: hubble_constant           = 0.704
